In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
import sys
sys.path.append("..")

In [43]:
from optimus import *

In [53]:
# Create optimus
op = Optimus()

In [5]:
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, IntegerType, ArrayType

df = op.create.df([
                ("words", "str", True),
                ("num", "int", True),
                ("animals", "str", True),
                ("thing", StringType(), True),
                ("second", "int", True),
                ("filter", StringType(), True)
            ],
            [
                ("  I like     fish  ", 1, "dog dog", "housé", 5 , "a"),
                ("    zombies", 2, "cat", "tv", 6, "b"),
                ("simpsons   cat lady", 2, "frog", "table", 7, "1"),
                (None, 3, "eagle", "glass", 8, "c")
                
            ])

df.show()

+-------------------+---+-------+-----+------+------+
|              words|num|animals|thing|second|filter|
+-------------------+---+-------+-----+------+------+
|  I like     fish  |  1|dog dog|housé|     5|     a|
|            zombies|  2|    cat|   tv|     6|     b|
|simpsons   cat lady|  2|   frog|table|     7|     1|
|               null|  3|  eagle|glass|     8|     c|
+-------------------+---+-------+-----+------+------+



In [7]:
df.dtypes

[('words', 'string'),
 ('num', 'int'),
 ('animals', 'string'),
 ('thing', 'string'),
 ('second', 'int'),
 ('filter', 'string')]

## Append row
### Spark
Not available in Spark. You need to create a dataframe and the union to append a row

In [8]:
df.rows().append(["this is a word",2, "this is an animal", "this is a thing", 64, "this is a filter"]).show()

+-------------------+---+-----------------+---------------+------+----------------+
|              words|num|          animals|          thing|second|          filter|
+-------------------+---+-----------------+---------------+------+----------------+
|  I like     fish  |  1|          dog dog|          housé|     5|               a|
|            zombies|  2|              cat|             tv|     6|               b|
|simpsons   cat lady|  2|             frog|          table|     7|               1|
|               null|  3|            eagle|          glass|     8|               c|
|     this is a word|  2|this is an animal|this is a thing|    64|this is a filter|
+-------------------+---+-----------------+---------------+------+----------------+



## Filter by type
### Spark 
Not available in Spark Vanilla.

### Pandas


In [9]:
df.rows().filter(df["num"]==1).show()

+-------------------+---+-------+-----+------+------+
|              words|num|animals|thing|second|filter|
+-------------------+---+-------+-----+------+------+
|  I like     fish  |  1|dog dog|housé|     5|     a|
+-------------------+---+-------+-----+------+------+



In [23]:
value= "filter"
isinstance(value, (str, int, float, bool))

True

In [40]:
df.rows().filter_by_dtypes("filter", "integer").show()

False
['filter']
filter
integer


Py4JJavaError: An error occurred while calling o350.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 10, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\opt\spark\spark-2.3.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 219, in main
  File "C:\opt\spark\spark-2.3.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 139, in read_udfs
  File "C:\opt\spark\spark-2.3.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 119, in read_single_udf
  File "C:\opt\spark\spark-2.3.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 59, in read_command
  File "C:\opt\spark\spark-2.3.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\serializers.py", line 170, in _read_with_length
    return self.loads(obj)
  File "C:\opt\spark\spark-2.3.1-bin-hadoop2.7\python\l

## Drop by type

In [1]:
df.rows().drop_by_dtypes("filter", "integer").show()

NameError: name 'df' is not defined

In [16]:
from optimus.functions import is_data_type as fbdt
df.rows().filter_by_dtypes(fbdt("filter", "integer") ).show()

str


AssertionError: The [True] columns do not exists in the DataFrame with the following columns ['words', 'num', 'animals', 'thing', 'second', 'filter']

In [17]:
df.filter(df['num']==1).show()

+-------------------+---+-------+-----+------+------+
|              words|num|animals|thing|second|filter|
+-------------------+---+-------+-----+------+------+
|  I like     fish  |  1|dog dog|housé|     5|     a|
+-------------------+---+-------+-----+------+------+



## Drop 
### Spark
Drop rows not implemented

In [18]:
df.rows().drop((df["num"]==2) | (df["second"]==5)).show()

+-----+---+-------+-----+------+------+
|words|num|animals|thing|second|filter|
+-----+---+-------+-----+------+------+
| null|  3|  eagle|glass|     8|     c|
+-----+---+-------+-----+------+------+



In [19]:
df.rows().drop_by_dtypes("filter", "int").show()

Py4JJavaError: An error occurred while calling o351.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 10, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\opt\spark\spark-2.3.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 219, in main
  File "C:\opt\spark\spark-2.3.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 139, in read_udfs
  File "C:\opt\spark\spark-2.3.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 119, in read_single_udf
  File "C:\opt\spark\spark-2.3.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 59, in read_command
  File "C:\opt\spark\spark-2.3.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\serializers.py", line 170, in _read_with_length
    return self.loads(obj)
  File "C:\opt\spark\spark-2.3.1-bin-hadoop2.7\python\l

In [39]:
from optimus.functions import abstract_udf as audf

def func_data_type(value, attr):
    return value >1


df.rows().drop(audf("num", func_data_type, "bool")).show()

+-------------------+---+-------+-----+------+------+
|              words|num|animals|thing|second|filter|
+-------------------+---+-------+-----+------+------+
|  I like     fish  |  1|dog dog|housé|     5|     a|
+-------------------+---+-------+-----+------+------+



In [21]:
df.rows().drop_na("num").show()

+-------------------+---+-------+-----+------+------+
|              words|num|animals|thing|second|filter|
+-------------------+---+-------+-----+------+------+
|  I like     fish  |  1|dog dog|housé|     5|     a|
|            zombies|  2|    cat|   tv|     6|     b|
|simpsons   cat lady|  2|   frog|table|     7|     1|
|               null|  3|  eagle|glass|     8|     c|
+-------------------+---+-------+-----+------+------+



In [25]:
df.rows().drop_duplicates("num").show()

['num']
+-------------------+---+-------+-----+------+------+
|              words|num|animals|thing|second|filter|
+-------------------+---+-------+-----+------+------+
|  I like     fish  |  1|dog dog|housé|     5|     a|
|               null|  3|  eagle|glass|     8|     c|
|            zombies|  2|    cat|   tv|     6|     b|
+-------------------+---+-------+-----+------+------+

